# An inner parallel cutting plane method for convex mixed-integer optimization problems 

## Summary of the document

This document is complementary to the paper "Granularity in mixed-integer nonlinear optimization". The intention is to make the numerical results transparent. The generated results for the IPCP, the feasibility pump as well as for B-Hyb are stored as pandas data-frames and are concatenated herein. 

The latex-tables of the article are produced by using the command print(dataframe.to_latex(float_format = '%.2f'))

In [10]:
import pandas as pd
import numpy as np
import pickle
def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

## First quick check

In [12]:
result = load_obj('result_overview')

In [14]:
result

,vars,constrs,time L,time SOR,obj,constr_value,time_bonmin,obj_bonmin
cvxnonsep_normcon20r,"(40, 10, 0)","(21, 20, 10, 10)",1.3052077293395996,0.0816507339477539,-14.6549272394,-inf,0.282246,-15.262740
cvxnonsep_normcon30r,"(60, 15, 0)","(31, 30, 15, 15)",1.8531317710876465,0.07815122604370117,-14.8031228018,-inf,0.186782,-22.712540
ex4,"(36, 25, 25)","(30, 25, 25, 0)",0,0.07811260223388672,-6.70152492653,-inf,1.555873,-6.864137
ex1223a,"(7, 4, 4)","(9, 4, 4, 0)",0,0.09380674362182617,6.06978727803,-inf,0.181516,4.579582
ringpack_10_1,"(70, 50, 50)","(385, 330, 330, 0)",0,0.6109938621520996,-4.17164353115092,-inf,2.234720,-8.343287
smallinvDAXr1b150-165,"(31, 30, 0)","(4, 1, 1, 1)",0.2511715888977051,0.07285046577453613,224.691303163,-124.027294155,14.190041,164.970025
